# Jorge Bueno Perez & Noam Shmuel

# Text Mining and Social Media Mining, final project 
# Sentiment analysis performed on the transcripts and titles of youtube videos related to the game Cyberpunk 2077

# Importing packages:

In [1]:
import os
from pytube import YouTube
import moviepy.editor as mp
import re
import speech_recognition as sr
from os import path
from pydub import AudioSegment
from youtubesearchpython import SearchVideos
import pandas as pd
import numpy as np

# 1) Extracting urls, description, duration and views:

### The `SearchVideos` function has a `limit of videos` to be found in each search, hence `17 different searchs` will be performed, such as `cyberpunk 2077 review`, `cyberpunk 2077 good`, `cyberpunk 2077 bad`...

### As all the social media content like youtube is `dynamic`, the videos found could change over the time. This `search was performed on 16/12/2020` (6 days after the game was released)

In [2]:
find = ["cyberpunk 2077 review",
        "cyberpunk 2077 good",
        "cyberpunk 2077 bad",
        "cyberpunk 2077 problem",
        "cyberpunk 2077 best",
        "cyberpunk 2077 recommended",
        "cyberpunk game review",
        "cyberpunk 2077 opinion",
        "cyberpunk game opinion",
        "cyberpunk 2077 analysis",
        "cyberpunk game analysis",
        "cyberpunk 2077 overview",
        "cyberpunk game overview",
       "cyberpunk 2077 ps4 review",
       "cyberpunk 2077 xbox review",
       "cyberpunk 2077 pc review",
       "cyberpunk 2077 ps5 review"]
df= pd.DataFrame(columns=['url', 'description', 'duration', 'views'])
for i in find:
    search = SearchVideos(i, offset = 1, mode = "list", max_results = 20).result()
    df= df.append(pd.DataFrame(search, columns=['id', 'ref', 'url', 'description', 'user', 'duration', 'views', 'img', 'ref1', 'ref2'])[['url', 'description', 'duration', 'views']])

### It is possible that the found videos are repeated, hence the `duplicated videos` are going to be `removed`:

In [3]:
df= df.drop_duplicates(subset=['url']) 

### Some extracted `videos are live`, also these are going to be `removed`:

In [4]:
df= df[df.duration != "LIVE"] 

In [7]:
df.head()

,url,description,duration,views
0,https://www.youtube.com/watch?v=0UiXx-_fL5U,Cyberpunk 2077 Review - The Best Unfinished Ga...,28:43,105676
1,https://www.youtube.com/watch?v=QcH9c_rJItE,Cyberpunk 2077 - Easy Allies Review,13:34,22384
2,https://www.youtube.com/watch?v=QfpoTjUz1BY,Cyberpunk 2077 for Xbox One and PlayStation 4 ...,3:49,922825
3,https://www.youtube.com/watch?v=FPzMhniHikQ,A Brutally Honest Review Of Cyberpunk 2077,10:36,18784
4,https://www.youtube.com/watch?v=ldhiHfg-_DA,Cyberpunk 2077 review,14:40,52561


In [8]:
df.to_csv("~/Desktop/Projects/Text mining/Audio analysis/urls.csv", index=False)

# 2) Downloading the videos from the urls:

In [9]:
urls= pd.read_csv("~/Desktop/Projects/Text mining/Audio analysis/urls.csv")
len(urls)

192

### The videos are going to be downloaded with `mp4 format`

In [10]:
urls_list = urls.url.to_list()
folder= "/Users/Lajobu/Desktop/Projects/Text mining/Audio analysis/Audios"
remove= []
download= []
for i in urls_list:
    try:
        yt= YouTube(i)
        t= yt.streams.filter(only_audio=True).all()
        t[0].download(folder, filename= str(i))
        download.append(i)
    except:
        remove.append(i)

### There was `one error` when downloading the videos, hence this should be removed from the list of videos

In [11]:
remove

['https://www.youtube.com/watch?v=5YD1gJ4dUgo']

In [12]:
urls= urls[~urls.url.isin(remove)]

# 3) Extracting the audio of the video:

### The audio of the video is going to be transformed to `wav format`

In [14]:
paths= []
for file in [n for n in os.listdir(folder) if re.search('mp4',n)]:
    full_path = os.path.join(folder, file)
    output_path = os.path.join(folder, os.path.splitext(file)[0] + '.wav')
    clip = mp.AudioFileClip(full_path) # disable if do not want any clipping
    clip.write_audiofile(output_path)
    paths.append(output_path)
    os.remove(full_path) 

chunk:   4%|▍         | 403/9741 [00:00<00:02, 4029.36it/s, now=None]

MoviePy - Writing audio in /Users/Lajobu/Desktop/Projects/Text mining/Audio analysis/Audios/httpswwwyoutubecomwatchv=08mTqdwiEZ8.wav


chunk:   3%|▎         | 357/13642 [00:00<00:03, 3442.49it/s, now=None]

MoviePy - Done.
MoviePy - Writing audio in /Users/Lajobu/Desktop/Projects/Text mining/Audio analysis/Audios/httpswwwyoutubecomwatchv=QNP0alPlhaU.wav


chunk:   2%|▏         | 384/19395 [00:00<00:04, 3835.36it/s, now=None]  

...

chunk:   3%|▎         | 396/14844 [00:00<00:03, 3957.15it/s, now=None]  

MoviePy - Done.
MoviePy - Writing audio in /Users/Lajobu/Desktop/Projects/Text mining/Audio analysis/Audios/httpswwwyoutubecomwatchv=OKo_Ysti9po.wav


chunk:   2%|▏         | 391/19546 [00:00<00:04, 3906.53it/s, now=None]  

MoviePy - Done.
MoviePy - Writing audio in /Users/Lajobu/Desktop/Projects/Text mining/Audio analysis/Audios/httpswwwyoutubecomwatchv=mRpdQF78-IA.wav


MoviePy - Done.


# 4) Performing speech recognition  

### Once the audio of the videos are extracted we are ready to `perform speech recognition` with `google speech recognition`, this process would allow to extract the `transcript` of the videos

### The `first 3 minutes` of each video will be extracted, and the `language` to be used is `en-US`, hence if any video is in `other language`, this will `not be used`, as per the `exception` of the below cell:

In [16]:
r = sr.Recognizer()
urls_list_ = urls.url.to_list()
transcript= []
rating= []
for i in range(len(paths)):
    with sr.AudioFile(paths[i]) as source:
        try:
            audio= r.record(source, duration= 180)               
            n= r.recognize_google(audio, language='en-US', show_all=True)
            transcript.append(''.join(str(val) for key, val in n.items()))
            rating.append(YouTube(urls_list_[i]).rating)
        except: 
            transcript.append(np.nan)
            rating.append(np.nan)

In [17]:
urls["transcript"] = transcript
urls["rating"] = rating

In [18]:
urls.dropna(subset=['transcript'], inplace=True)

### After the above mentioned process `136 transcripts of videos` were extracted

In [19]:
len(urls)

136

In [20]:
urls.head()

,url,description,duration,views,transcript,rating
0,https://www.youtube.com/watch?v=0UiXx-_fL5U,Cyberpunk 2077 Review - The Best Unfinished Ga...,28:43,105676,[{'transcript': 'so I get a lot of questions a...,4.864700
1,https://www.youtube.com/watch?v=QcH9c_rJItE,Cyberpunk 2077 - Easy Allies Review,13:34,22384,"[{'transcript': ""how does cyberpunk 2077 on th...",4.870146
2,https://www.youtube.com/watch?v=QfpoTjUz1BY,Cyberpunk 2077 for Xbox One and PlayStation 4 ...,3:49,922825,"[{'transcript': ""this is a real blood of my cy...",4.836886
3,https://www.youtube.com/watch?v=FPzMhniHikQ,A Brutally Honest Review Of Cyberpunk 2077,10:36,18784,"[{'transcript': ""what is going on your nation ...",4.854014
5,https://www.youtube.com/watch?v=z1rUP17fayA,Cyberpunk 2077 PC Review (With New Gameplay),9:00,1306680,[{'transcript': 'what the what the hell are yo...,4.627821


### As it can be seen above, the `transcript` column also always includes the text `transcript'` and `confidence'`, this could be `removed with regex`, however for the purpose of the sentiment analysis it is not neccesary, as the words can be omitted within the function `stopwords.add()`

In [21]:
#urls.to_csv("~/Desktop/Projects/Text mining/Audio analysis/cyberpunk.csv", index=False)